In [ ]:

import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import weightedstats as ws


# consider using this for weighted stats: http://www.ccgalberta.com/pygeostat/welcome.html



## Functions and tools used in the analysis

In [ ]:
def weighted_mean(var, wts):
    """Calculates the weighted mean"""
    return np.average(var, weights=wts)

def weighted_median_series(val, weight):
    """Calculates the weighted median
    ArithmeticError
    If the sum of the weights is zero, or if the weights are not positive.
    """
    try:
        df = pd.DataFrame({"val": val, "weight": weight})
        df_sorted = df.sort_values("val")
        cumsum = df_sorted["weight"].cumsum()
        cutoff = df_sorted["weight"].sum() / 2.
        result = df_sorted[cumsum >= cutoff]["val"].iloc[0]
        # return just the value
    except:
        result = np.nan
    return result


def weighted_median(df, val_column, weight_column):
    """Calculates the weighted median
    ArithmeticError
    If the sum of the weights is zero, or if the weights are not positive.
    """
    df_sorted = df.sort_values(val_column)
    cumsum = df_sorted[weight_column].cumsum()
    cutoff = df_sorted[weight_column].sum() / 2.
    return df_sorted[cumsum >= cutoff][val_column].iloc[0]

def weighted_variance(var, wts):
    """Calculates the weighted variance"""
    return np.average((var - weighted_mean(var, wts))**2, weights=wts)


def weighted_skew(var, wts):
    """Calculates the weighted skewness"""
    return (np.average((var - weighted_mean(var, wts))**3, weights=wts) /
            weighted_variance(var, wts)**(1.5))

def weighted_kurtosis(var, wts):
    """Calculates the weighted skewness"""
    return (np.average((var - weighted_mean(var, wts))**4, weights=wts) /
            weighted_variance(var, wts)**(2))



def run_weighted_median_on_grouped_df(df, groupby_column, value_column, weight_column):
    """Calculate the weighted median of a dataframe grouped by a column.
    Args:
        df (pandas.DataFrame): DataFrame to calculate weighted median on.
        groupby_column (str): Column to group by.
        value_column (str): Column to calculate weighted median on.
        weight_column (str): Column to use as weight.
    Returns:
        pandas.DataFrame: DataFrame with weighted median for each group.
    """
    # initialize empty list
    d = []
    # loop through each group
    for i in df[groupby_column].unique():
        df_group = df[df[groupby_column] == i]
        # if rows in dataframe are more than 1, calculate weighted median
        if len(df_group) > 1:
            median = weighted_median(df_group, value_column, weight_column)
        else:
            median = df_group[value_column].values[0]
        d.append(
            {
                groupby_column: i,
                "median": median,
            }
        )
    return pd.DataFrame(d)


def load_data(data_file: str) -> pd.DataFrame:
    '''
    Load data from /data directory
    '''
    PATH = pathlib.Path().resolve()
    DATA_PATH = PATH.joinpath("../data").resolve()
    return pd.read_csv(DATA_PATH.joinpath(data_file))



## Import Data from CSVs.
CSVs created in previous script, which did the cycling mobility on a per country basis

In [ ]:


df_zones_input= load_data("GIS/points_5min_15min_dtw_csv.csv")
# import df_input and set index as alpha 3 
df_input = load_data("country_data_with_cycling_distance.csv")

## manual adjustment of some columns
df_zones_input["dtw_1"] = df_zones_input["dtw_1"] / 1000 # turn to kms
df_zones_input["max distance walking"] = 3.0 # temp, set max distance reachbale for all countriesl, will change to be bespoke soon
df_input["max distance cycling"] = 8 * df_input["trip_velocity_mean"] # temp, set max distance reachbale for all countriesl, will change to be bespoke soon

## Merge the dataframes
Do some manual adjustments to populations

In [ ]:

# this analysis loses some data as the overlap between the rasters is not perfect. To reduce this error, use the 30 arc second data. Too much heavy lifting for my computer to do this at the moment.

#merge df_input and df_zones on ISO_CC. This assigns all the country data to each zone.
# join inner will remove some of the data that is not in both datasets
df_zones = df_zones_input.merge(df_input, left_on="ISOCODE", right_on="alpha3", how="inner")


#adjust population to account for 9 values per raster point (2.5 to 5 arc min resoltuions. 9 values per point)
df_zones["AdjPopFloat"] = df_zones["pop_count_15_1"] / 9 

# # convert population density to percent of national population on a per country basis, grouped by ISO_CC
df_zones["pop_density_perc"] = df_zones.groupby("ISOCODE")["AdjPopFloat"].apply(lambda x: x / x.sum())

# multiply population density by population on a per country basis
df_zones["pop_zone"] = df_zones["pop_density_perc"] * df_zones["Population"]


In [ ]:
# sum the population in each zone
df_zones["country_pop_raw"] = df_zones.groupby("ISOCODE")["pop_zone"].transform("sum")
df_zones["country_pop_ratio"] = df_zones.groupby("ISOCODE")["AdjPopFloat"].transform("sum")


## The section below calculates the population per zone that can reach water

Consider putting in household size and bike sharing here

In [ ]:
df_input.head()

In [ ]:
# population with piped water
df_zones["zone_pop_piped"] = df_zones["pop_zone"]*df_zones["Nat Piped"]/100
# calculaye unpiped, this is not "un piped" from the dataset, but rather the population that does not have piped water
# unpiped from the dataset refers to ONLY inproved unpiped. So do 100 - piped to get the total unpiped (inlcuding unimproved water sources)
df_zones["zone_pop_unpiped"] = df_zones["pop_zone"]*(100-df_zones["Nat Piped"])/100


# is it possible to reach water with walking/cycling
df_zones["zone_cycling_okay"] = (df_zones["dtw_1"] < df_zones["max distance cycling"])*1 # multiply by 1 to force to binary not true/false
df_zones["zone_walking_okay"] = (df_zones["dtw_1"] < df_zones["max distance walking"])*1

# how many people can collect water in the zone
df_zones["fraction_of_zone_with_cycling_access"] = df_zones["zone_cycling_okay"]* (df_zones["PBO"])/100
df_zones["fraction_of_zone_with_walking_access"] = df_zones["zone_walking_okay"] * 1

#
df_zones["population_piped_with_cycling_access"] = df_zones["fraction_of_zone_with_cycling_access"] * df_zones["zone_pop_piped"]
df_zones["population_piped_with_walking_access"] = df_zones["fraction_of_zone_with_walking_access"] * df_zones["zone_pop_piped"]

# select the maximum between the two
df_zones["population_piped_with_access"] = df_zones[["population_piped_with_cycling_access", "population_piped_with_walking_access"]].max(axis=1)

# zone pop without water
df_zones["zone_pop_with_water"] =  df_zones["population_piped_with_access"] + df_zones["zone_pop_unpiped"]
df_zones["zone_pop_without_water"] = df_zones["pop_zone"] - df_zones["zone_pop_with_water"]




## Group the zones in to country groups
Use groupby agg and then apply to summarise the zone data in to country level, then create some summary columns

In [ ]:
# use groupby to create dataframe of country level data from df_zones
df_countries = df_zones.groupby("ISOCODE").agg({
    "Entity":"first",
    "country_pop_raw": "first",
    "zone_pop_with_water":"sum",
    "zone_pop_without_water":"sum",
    "population_piped_with_access":"sum"  ,
    "population_piped_with_cycling_access":"sum",
    "population_piped_with_walking_access":"sum",
    "Nat Piped":"first",
    "region":"first",
    # call the weighted median function on the column
}).reset_index()

# use groupby to create weighted median, needs to be speerate from the above groupby as it uses apply, which can't be used in the same groupby
# needs to use apply because the function required two columns as input
df_median_group = df_zones.groupby(['ISOCODE']).apply(lambda x : pd.Series({'weighted_med':weighted_median(x,"dtw_1","pop_zone")}))

# merge the weighted median back into the df_countries dataframe
df_countries = df_countries.merge(df_median_group, on="ISOCODE")

# drop rows from the dataframe that have Nan in pop_zone and dtw_1
df_zones = df_zones.dropna(subset=["pop_zone", "dtw_1"])

# create summary columns
#rename zone columns to country
df_countries = df_countries.rename(columns={"zone_pop_with_water":"country_pop_with_water", "zone_pop_without_water":"country_pop_without_water"})
# create percent
df_countries["percent_with_water"] = df_countries["country_pop_with_water"] / df_countries["country_pop_raw"] * 100
df_countries["percent_without_water"] = df_countries["country_pop_without_water"] / df_countries["country_pop_raw"] * 100

Clean Up Data

In [43]:
# Clean up data from spurious country values
# uses libya as the max possible diatnce to water (from Kummu paper results)

# remove any nan rows
df_countries = df_countries.dropna()


# first pass clean up

# remove any rows where the median is more than 1km more than "LBY" (Libya)'s median
max_distance = df_countries.loc[df_countries["ISOCODE"] == "LBY", "weighted_med"].values[0] + 1
countries_further_than_libya = df_countries[df_countries["weighted_med"] > max_distance]
df_countries = df_countries[df_countries["weighted_med"] < max_distance]

####### Manual remove countries ###############
# inspecting the results, there are a few countries that are clearly wrong, remove them manually
list_of_countries_to_remove = ["GUM", "ASM", "TON", "MNP", "ATG", "DMA", "ABW", "BRB"]
df_of_countries_to_remove = df_countries[df_countries["ISOCODE"].isin(list_of_countries_to_remove)]
df_countries = df_countries[~df_countries["ISOCODE"].isin(list_of_countries_to_remove)]
##############################


# summary of removed countries
print("Countries removed from analysis")
print(countries_further_than_libya[["Entity", "weighted_med"]])
print("Countries removed manually")
print(df_of_countries_to_remove[["Entity", "weighted_med"]])






Countries removed from analysis
Empty DataFrame
Columns: [Entity, weighted_med]
Index: []
Countries removed manually
                 Entity  weighted_med
9   Antigua and Barbuda     56.080301
52             Dominica     69.260383
0                 Aruba     87.800906
27             Barbados    144.138797


# Plotting and Visualising

In [44]:
# create choropleth map of population with water from df_country

hover_data_list =[
    "Entity",
    "country_pop_raw",
    "country_pop_with_water",
    "country_pop_without_water",
    "population_piped_with_access",
    "population_piped_with_cycling_access",
    "population_piped_with_walking_access",
    "percent_without_water",
    "percent_with_water",
    "Nat Piped",
    "region",
    "weighted_med"
    ]


choro = px.choropleth(
    title="Percent of Population Has to Relocate",
    data_frame=df_countries,
    locations="ISOCODE",
    height=600,
    color="percent_without_water",
    hover_name="Entity",
    hover_data=hover_data_list
)
choro.layout.coloraxis.colorbar.title = ''
choro.show()


In [45]:
# create bubble chart from df_countries comprising: access to water, piped water, and using population as the size of the bubble
# create a new column for the size of the bubble
df_countries["bubble_size"] = df_countries["country_pop_raw"] / 1000000 +50

# create a new column for the color of the bubble
df_countries["bubble_color"] = df_countries["Nat Piped"]

# create a new column for the text of the bubble
df_countries["bubble_text"] = df_countries["Entity"]

px.scatter(df_countries, x="percent_without_water", y="Nat Piped", size="bubble_size", color="region", hover_name="bubble_text", title="Access to Water vs. Piped Water vs. Population")

Above graph is the money shot. NMeed to fix bubble sizes, maybe add labels

In [46]:
#sort the countries by distance to water, plot the results
df_countries.sort_values(by="weighted_med", inplace=True)
px.line(df_countries, x="Entity", y="weighted_med", color="region", title="Distance to Water")


In [47]:
# plot the countries by population withgout water and percent without water
px.scatter(df_countries, x="percent_without_water", y="country_pop_without_water", size="bubble_size", color="region", hover_name="bubble_text", title="Population vs. Percent Without Water")


In [48]:
# summary statistics of zones by for slected country
# select country
country = "CAF"

# interertsinbg ciolumns
interesting_cols =[
    "Entity",
    "dtw_1",
    "pop_zone",
    "country_pop_raw",
    "population_piped_with_access",
    "population_piped_with_cycling_access",
    "population_piped_with_walking_access",
    "Nat Piped",
    "region",
]

other_cols = [
    "zone_pop_unpiped",
    "zone_pop_piped",
    "pop_zone"

]


# create dataframe of zones for selected country
df_zones_country = df_zones[df_zones["ISOCODE"] == country]
df_zones_country = df_zones_country[other_cols]

# only keep intersting columns




# create summary statistics
df_zones_country.describe()







,zone_pop_unpiped,zone_pop_piped,pop_zone
count,7295.000000,7295.000000,7295.000000
mean,580.012175,94.420587,674.432762
std,2720.028016,442.795258,3162.823274
min,4.751983,0.773579,5.525561
25%,68.363462,11.128936,79.492397
50%,226.394859,36.854977,263.249836
75%,667.920466,108.731239,776.651705
max,85345.360830,13893.430833,99238.791662


In [50]:
# create tree graph of population wihtout water by entity from df_countries...
#TBC



